<a href="https://colab.research.google.com/github/stephen1hong/AI-LLMs/blob/master/Chinese_Llama_2_7b_4bit_Inference_ColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

在Colab上使用免费T4 GPU进行Chinese-Llama-2-7b-4bit推理

参考链接:
在Colab上使用免费T4 GPU进行Chinese-Llama-2-7b-4bit推理: https://www.cnblogs.com/justLittleStar/p/17745291.html

[1] Llama-2-7b-4bit推理 https://www.bilibili.com/read/cv25258378/

[2] 原始Kaggle Notebook链接：https://www.kaggle.com/code/tiansztianszs/chinese-llama-2-7b-4bit/notebook

In [1]:
!pip install --upgrade accelerate
!pip install bitsandbytes transformers_stream_generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install transformers
!pip install sentencepiece
!pip install torch
!pip install accelerate

注意此时，安装完accelerate后需要重启notebook，不然报错误

In [1]:
import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer


In [2]:
model_path = "LinkSoul/Chinese-Llama-2-7b-4bit"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        load_in_4bit=True,
        torch_dtype=torch.float16,
        device_map='auto'
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [4]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

instruction = """[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\n{} [/INST]"""

In [5]:
prompt = instruction.format("What is the meaning of life")
generate_ids = model.generate(tokenizer(prompt, return_tensors='pt').input_ids.cuda(), max_new_tokens=4096, streamer=streamer)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


The meaning of life is a philosophical question that has been debated for centuries. There is no one definitive answer, as different people and cultures may have different beliefs and values. Some people believe that the meaning of life is to seek happiness, while others believe that it is to fulfill a higher purpose or to serve a greater good. Ultimately, the meaning of life is a personal and subjective question that each individual must answer for themselves. 


In [6]:
prompt = instruction.format("如何做个不拖延的人")
generate_ids = model.generate(tokenizer(prompt, return_tensors='pt').input_ids.cuda(), max_new_tokens=4096, streamer=streamer)

答案：不拖延的人是一个很好的目标，但是要成为一个不拖延的人并不容易。以下是一些建议，可以帮助你成为一个不拖延的人：

1. 制定计划：制定一个详细的计划，包括每天要完成的任务和时间表。这样可以帮助你更好地组织时间，并避免拖延。
2. 设定目标：设定一个明确的目标，并制定一个实现这个目标的计划。这样可以帮助你更好地了解自己的目标，并更有动力地去完成任务。
3. 克服拖延的心理：拖延的心理是一个常见的问题，但是可以通过一些方法克服。例如，你可以尝试使用一些技巧来克服拖延，如分解任务、使用时间管理工具等。
4. 坚持自己的计划：坚持自己的计划是非常重要的。如果你经常拖延，那么你需要坚持自己的计划，并尽可能地按照计划去完成任务。
5. 寻求帮助：如果你拖延的问题严重，那么你可以寻求帮助。例如，你可以寻求心理咨询师的帮助，或者寻求朋友或家人的帮助。

希望这些建议能帮助你成为一个不拖延的人。 
